In [2]:
# OS Libaries
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [1]:
class HMM:
    """
    
    alpha (Joint probability): represents the joint probability of observing all of the given data up to time t and the value of z
    --> The forward variable
    --> alpha_t(i) = P(O_1 0_2 ... 0_T, s_t=i | theta) 
    
    beta  (conditional prob) : the probability of all future data from time t+1 until to T given the value of the state_t = i
    --> The backward variable
    --> B_t(i) = P(O_t+1 O_t+2 ... O_T | s_t=i, theta)
    
    gamma (conditional prob) : probability of being in state j at time t, given the oberservation sequence and the model
    --> y_t(i) = P(s_t=i | x_1:T, theta)
    
    """
    def __init__(self, num_latent_states, observations):
        self.num_latent_states      = num_latent_states
        self.T                      = len(observations)
        self.num_observation_states = len(set(observations))
        
        self.transition_prob      = np.zeros([self.num_latent_states, self.num_latent_states])
        self.emission_prob        = np.zeros([self.num_latent_states, self.num_observation_states])
        self.initial_distribution = np.zeros(self.num_latent_states)
        
        self.alpha = np.zeros([self.T, self.num_latent_states]) # alpha_t(i) = P(x_1:t, s_t=i)
        self.beta  = np.zeros([self.T, self.num_latent_states]) # beta_t(i) = P(x_t+1:T | s_t=i)
        self.gamma = np.zeros([self.T, self.num_latent_states]) # gamma_t(i) = P(s_t=i | x_1:T)
        self.xi    = np.zeros([self.T, self.num_latent_states, self.num_latent_states]) # xi_t(i,j) = P(s_t=i, s_t+1=j | x_1:T)
    
    # Initialise model before training it    
    def random_init(self, seed=False):
        if seed:
            np.random.seed(seed)
        
        # initialise transition prob
        for i in range(self.num_latent_states):
            self.transition_prob[i,:] = np.random.uniform(low=0, high=1, size=self.num_latent_states)
            self.transition_prob[i,:] /= np.sum(self.transition_prob[i,:])
        
        # initialise emission_prob
        for i in range(self.num_latent_states):
            self.emission_prob[i,:] = np.random.uniform(low=0, high=1, size=self.num_observation_states)
            self.emission_prob[i,:] /= np.sum(self.emission_prob[i,:])
            
        self.initial_distribution = np.random.uniform(low=0, high=1, size=self.num_latent_states)
        self.initial_distribution /= np.sum(self.initial_distribution)
    
    def Baum_Welch_E_step(self):
        # computes alpha, beta, gamma, xi
        # forward-backward algorithm
        
        # forward pass
        for i in range(self.num_latent_states):
            self.alpha[0, i] = self.initial_distribution[i] * self.emission_prob[i, state_idx[X[0]]]
        for t in range(1, self.T):
            for i in range(self.num_latent_states):
                self.alpha[t, i] = np.sum([self.alpha[t-1, j] * self.transition_prob[j,i] for j in range(self.num_latent_states)]) * self.emission_prob[i, state_idx[X[t]]]
        
        # backward pass
        self.beta[self.T-1, :] = np.ones((1,self.num_latent_states))
        for t in range(self.T-2, -1, -1):
            for i in range(self.num_latent_states):
                self.beta[t, i] = np.sum([self.emission_prob[j, state_idx[X[t+1]]] * self.transition_prob[i, j] * self.beta[t+1, j]for j in range(self.num_latent_states)])
            
        # marginal
        for t in range(self.T):
            for j in range(self.num_latent_states):
                self.gamma[t,j] = self.alpha[t,j] * self.beta[t,j] / np.sum([self.alpha[t,k]*self.beta[t,k] for k in range(self.num_latent_states)])
        
        # xi
        for t in range(self.T-1):
            for i in range(self.num_latent_states):
                for j in range(self.num_latent_states):
                    self.xi[t,i,j] = self.alpha[t,i] * self.transition_prob[i,j] * self.emission_prob[j, state_idx[X[t+1]]]*self.beta[t+1, j]
                    self.xi[t,i,j] /= np.sum([np.sum([self.alpha[t,i] * self.transition_prob[i,j] *self.emission_prob[j, state_idx[X[t+1]]]*self.beta[t+1,j] for j in range(self.num_latent_states)]) for i in range(self.num_latent_states)])
                
    def Baum_Welch_M_step(self):
        # computes pi, transition prob, emission prob
        indicator = lambda x, y: 1 if x == y else 0
        for i in range(self.num_latent_states):
            self.initial_distribution[i] = self.gamma[0,i]
            for j in range(self.num_latent_states):
                self.transition_prob[i,j] = np.sum([self.xi[t, i, j] for t in range(self.T)])/np.sum([self.gamma[t,i] for t in range(self.T)])
            for k in range(self.num_observation_states):
                self.emission_prob[i,k] = np.sum([self.gamma[t,i] * indicator(k, state_idx[X[t]]) for t in range(self.T)]) / np.sum([self.gamma[t,i] for t in range(self.T)])
        
    def show_params(self):
        print(f'    Initial distribution: \n    {self.initial_distribution}')
        print(f'    Transition probabilities: \n    {self.transition_prob}')
        print(f'    Emission probabilities: \n    {self.emission_prob}')
        
    def train(self, num_iter):
        print('Initial parameters:')
        self.show_params()
        for i in range(num_iter):
            self.Baum_Welch_E_step()
            self.Baum_Welch_M_step()
            if (i+1) % 10 == 0:
                print('\n\n')
                print(f'#### Iteration {i+1} complete ####')
                self.show_params()

In [ ]:
X = 'AABBBACABBBACAAAAAAAAABBBACAAAAABACAAAAAABBBBACAAAAAAAAAAAABACABACAABBACAAABBBBACAAABACAAAABACAABACAAABBACAAAABBBBACABBACAAAAAABACABACAAABACAABBBACAAAABACABBACA'
state_idx = {'A':0, 'B':1, 'C':2}

model = HMM(num_latent_states=4, observations=X)
model.random_init()
model.train(num_iter=50)

Initial parameters:
    Initial distribution: 
    [0.23076425 0.27165265 0.26073199 0.23685111]
    Transition probabilities: 
    [[0.27586823 0.13151764 0.1121123  0.48050183]
 [0.1657152  0.29840698 0.36808037 0.16779745]
 [0.21692735 0.12157569 0.30177898 0.35971798]
 [0.57208734 0.0139302  0.31096729 0.10301517]]
    Emission probabilities: 
    [[0.20432574 0.34820151 0.44747275]
 [0.24785536 0.30446154 0.4476831 ]
 [0.29300753 0.22757012 0.47942234]
 [0.6769375  0.00602381 0.31703869]]



#### Iteration 10 complete ####
    Initial distribution: 
    [2.03627857e-03 4.58344896e-07 3.65289089e-02 9.61434354e-01]
    Transition probabilities: 
    [[0.03106268 0.14655377 0.03032431 0.79147838]
 [0.00975048 0.54120515 0.01862859 0.43005557]
 [0.05929699 0.28780444 0.12938368 0.52185365]
 [0.66665479 0.01167175 0.26938799 0.03763746]]
    Emission probabilities: 
    [[4.18724278e-01 1.75903712e-01 4.05372010e-01]
 [3.01658404e-02 9.69833794e-01 3.65836821e-07]
 [6.38062324e-01 2.3